In [1]:
import pandas as pd
from numpy.random import rand
from yaml import load, Loader

from planktonsdm.predict import predict
from planktonsdm.functions import example_data

In [2]:
#If you want to debug the package without repackaging, you can read files locally:

#import sys
#sys.path.insert(0, '/home/phyto/planktonSDM/planktonsdm/')
#from predict import predict
#from functions import example_data

In [3]:
X, y = example_data(y_name =  "Coccolithus pelagicus", n_samples=500, n_features=5, noise=20, random_state=model_config['seed'])

envdata = pd.DataFrame({"no3": rand(50), "mld": rand(50), "par": rand(50), "o2": rand(50), "temp": rand(50),
                        "lat": range(0,50, 1), "lon": 50*[1]})
envdata.set_index(['lat', 'lon'], inplace=True)
envdata = pd.DataFrame(X)

In [4]:
with open('/home/phyto/planktonSDM/configuration/1-phase_rf.yml', 'r') as f:
    model_config = load(f, Loader=Loader)

m = predict(X, y, envdata, model_config)
m.make_prediction()

scale X = True
scaled X_predict

number of models in ensemble:3
predicting classifier
exporting rf prediction to: /home/phyto/CoccoML/ModelOutput/rf/predictions/
predicting classifier
exporting xgb prediction to: /home/phyto/CoccoML/ModelOutput/xgb/predictions/
predicting classifier
exporting knn prediction to: /home/phyto/CoccoML/ModelOutput/knn/predictions/
predicting ensemble model
exporting ensemble prediction to: /home/phyto/CoccoML/ModelOutput/ens/predictions/
exporting ensemble scores to: /home/phyto/CoccoML/ModelOutput/ens/scoring/
finished
execution time: 3.619521379470825 seconds


In [5]:
with open('/home/phyto/planktonSDM/configuration/2-phase_ensemble.yml', 'r') as f:
    model_config = load(f, Loader=Loader)
m = predict(X, y, envdata, model_config)
m.make_prediction()

scale X = True
scaled X_predict

number of models in ensemble:1
predicting classifier
finished
execution time: 0.06909871101379395 seconds
